In [3]:
import csv
import os
import re
import unicodedata
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# === Chrome Setup ===
options = Options()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("--headless")  # Optional: for headless execution

driver_path = r'F:\SaylaniPython\Selenium Driver\chromedriver-win64\chromedriver.exe'
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

# === CSV Setup ===
csv_file = "tradingview_nyse_a.csv"

# === Text Cleaner ===
def clean_text(text):
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ascii", "ignore").decode("utf-8")
    text = re.sub(r'[^a-zA-Z0-9.%+:/,\s-]', '-', text.strip())
    return text

try:
    url = "https://www.tradingview.com/symbols/NYSE-A/"
    driver.get(url)
    WebDriverWait(driver, 15).until(lambda d: d.execute_script("return document.readyState") == "complete")

    # Wait for boxes to load
    WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".apply-overflow-tooltip.value-QCJM7wcY"))
    )

    # Get all boxes
    boxes = driver.find_elements(By.CSS_SELECTOR, ".apply-overflow-tooltip.value-QCJM7wcY")

    labels = []
    values = []

    for i in range(min(8, len(boxes))):  # Only first 8
        label_element = boxes[i].find_element(By.XPATH, "./../../preceding-sibling::*[1]")  # Navigate to label
        label = clean_text(label_element.text)
        value = clean_text(boxes[i].text)
        labels.append(label)
        values.append(value)

    # Save to CSV (headings as columns)
    with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(labels)
        writer.writerow(values)

    print(f"✅ Successfully saved {len(labels)} columns to: {csv_file}")

except Exception as e:
    print(f"❌ Error: {e}")

finally:
    driver.quit()


✅ Successfully saved 8 columns to: tradingview_nyse_a.csv
